**Given:**
  - Two 2D points ( $( \mathbf{x_1} = (x_1, y_1) )$ and $( \mathbf{x_2} = (x_2, y_2) )$ ) from different images.
  - Intrinsic camera matrices $( \mathbf{K_1} )$ and $( \mathbf{K_2} )$.
  - Camera pose (Rotation $( \mathbf{R} )$ and translation $( \mathbf{t} )$ between the cameras).

**Objective:**
  - Reconstruct the corresponding 3D point $( \mathbf{X} )$.

In [1]:
import numpy as np

**Normalized Image Points:** Convert pixel coordinates to normalized camera coordinates (coordinates in the camera's own coordinate system).
  
  $$
  \tilde{\mathbf{x}} = \mathbf{K}^{-1} \begin{pmatrix} x \\ y \\ 1 \end{pmatrix}
  $$

In [2]:
def normalize_image_point(x, K):
    """
    Normalize image point using the intrinsic matrix.
    x: (x, y) image point
    K: Intrinsic camera matrix
    Returns normalized image point in homogeneous coordinates
    """
    x_homog = np.array([x[0], x[1], 1.0])
    x_norm = np.linalg.inv(K) @ x_homog
    return x_norm

Compute the projection matrices $( \mathbf{P_1} )$ and $( \mathbf{P_2} )$ for the two cameras.

**Explanation:**

- **First Camera (Reference Frame):**

  $$
  \mathbf{P}_1 = \mathbf{K}_1 \left[ \mathbf{I} \ | \ \mathbf{0} \right]
  $$

- **Second Camera:**

  $$
  \mathbf{P}_2 = \mathbf{K}_2 \left[ \mathbf{R} \ | \ \mathbf{t} \right]
  $$

In [4]:
def compute_projection_matrix(K, R=np.eye(3), t=np.zeros((3, 1))):
    """
    Compute the projection matrix P = K [R | t]
    K: Intrinsic matrix
    R: Rotation matrix (3x3)
    t: Translation vector (3x1)
    """
    RT = np.hstack((R, t.reshape(3, 1)))
    P = K @ RT
    return P

Set up a linear system $( \mathbf{A} \mathbf{X} = 0 )$ to solve for the 3D point $( \mathbf{X} )$.

**Explanation:**

- For each camera, we have:

  $$[
  \mathbf{x} \times (\mathbf{P} \mathbf{X}) = 0
  ]$$

- The cross product leads to two equations per image point (since the homogeneous scale can be absorbed).

- Stack the equations from both images to form an $( 4 \times 4 )$ linear system.

In [5]:
def construct_linear_system(x1_norm, x2_norm, P1, P2):
    """
    Construct the linear system A X = 0
    x1_norm, x2_norm: Normalized image points in homogeneous coordinates
    P1, P2: Projection matrices
    Returns matrix A
    """
    A = np.zeros((4, 4))
    # First image
    A[0] = x1_norm[0] * P1[2] - P1[0]
    A[1] = x1_norm[1] * P1[2] - P1[1]
    # Second image
    A[2] = x2_norm[0] * P2[2] - P2[0]
    A[3] = x2_norm[1] * P2[2] - P2[1]
    return A

Use Singular Value Decomposition (SVD) to solve the homogeneous system $( \mathbf{A} \mathbf{X} = 0 )$.

In [7]:
def solve_linear_system(A):
    """
    Solve the linear system A X = 0 using SVD
    Returns X in homogeneous coordinates
    """
    U, S, Vt = np.linalg.svd(A)
    X = Vt[-1]
    return X

In [ ]:
def dehomogenize(X):
    """
    Convert homogeneous coordinates to 3D coordinates
    X: Homogeneous coordinates (4 elements)
    Returns 3D coordinates (3 elements)
    """
    return X[0:3] / X[3]

In [ ]:
def triangulate_point(x1, x2, K1, K2, R, t):
    """
    Triangulate a 3D point from two 2D image points.
    x1, x2: Image points in pixel coordinates (tuples or lists of (x, y))
    K1, K2: Intrinsic camera matrices
    R: Rotation matrix from camera 1 to camera 2
    t: Translation vector from camera 1 to camera 2
    Returns 3D point coordinates
    """
    # Step 1: Normalize image points
    x1_norm = normalize_image_point(x1, K1)
    x2_norm = normalize_image_point(x2, K2)
    
    # Step 2: Compute projection matrices
    P1 = compute_projection_matrix(K1)
    P2 = compute_projection_matrix(K2, R, t)
    
    # Step 3: Set up the linear system
    A = construct_linear_system(x1_norm, x2_norm, P1, P2)
    
    # Step 4: Solve for X using SVD
    X_homog = solve_linear_system(A)
    
    # Step 5: Dehomogenize coordinate
    X = dehomogenize(X_homog)
    return X

In [8]:
import numpy as np

def normalize_image_point(x, K):
    x_homog = np.array([x[0], x[1], 1.0])
    x_norm = np.linalg.inv(K) @ x_homog
    return x_norm

def compute_projection_matrix(K, R=np.eye(3), t=np.zeros((3, 1))):
    RT = np.hstack((R, t.reshape(3, 1)))
    P = K @ RT
    return P

def construct_linear_system(x1_norm, x2_norm, P1, P2):
    A = np.zeros((4, 4))
    A[0] = x1_norm[0] * P1[2] - P1[0]
    A[1] = x1_norm[1] * P1[2] - P1[1]
    A[2] = x2_norm[0] * P2[2] - P2[0]
    A[3] = x2_norm[1] * P2[2] - P2[1]
    return A

def solve_linear_system(A):
    U, S, Vt = np.linalg.svd(A)
    X = Vt[-1]
    return X

def dehomogenize(X):
    return X[0:3] / X[3]

def triangulate_point(x1, x2, K1, K2, R, t):
    x1_norm = normalize_image_point(x1, K1)
    x2_norm = normalize_image_point(x2, K2)
    P1 = compute_projection_matrix(K1)
    P2 = compute_projection_matrix(K2, R, t)
    A = construct_linear_system(x1_norm, x2_norm, P1, P2)
    X_homog = solve_linear_system(A)
    X = dehomogenize(X_homog)
    return X

# Example intrinsic parameters
fx = fy = 800  # Focal length in pixels
cx = cy = 640  # Principal point

K1 = np.array([[fx,  0, cx],
               [ 0, fy, cy],
               [ 0,  0,  1]])
K2 = K1.copy()

# Example rotation and translation
theta = np.deg2rad(10)  # Rotation around y-axis by 10 degrees
R = np.array([[ np.cos(theta), 0, np.sin(theta)],
              [            0, 1,            0],
              [-np.sin(theta), 0, np.cos(theta)]])

t = np.array([1, 0, 0])  # Translation along x-axis

# Example image points
x1 = (650, 500)  # Point in image 1
x2 = (620, 480)  # Corresponding point in image 2

# Triangulate the point
X = triangulate_point(x1, x2, K1, K2, R, t)

print("Reconstructed 3D point:", X)

Reconstructed 3D point: [ 2.73769856  2.93010083 -3.44638   ]
